In [1]:
import pandas as pd

def combine(dataset, combine):
    dataset.loc[(dataset.Offense.str.contains('NO ') == True), 'Offense']= 'N'
    
    dataset.loc[(dataset.Reported_As.str.contains('SEXUAL') == True), 'Offense'] = 'SEXUAL'
    dataset.loc[(dataset.Offense.str.contains('RAPE') == True), 'Offense']= 'SEXUAL'
    
    dataset.loc[(dataset.Offense.str.contains('HOMICIDE') == True), 'Offense']= 'MURDER'
    dataset.loc[(dataset.Offense.str.contains('MANSLAUGHTER') == True), 'Offense']= 'MURDER'
    
    for x in combine:
        dataset.loc[(dataset.Offense.str.contains('NO '+ x) == True), 'Offense']= 'N'
        dataset.loc[(dataset.Offense.str.contains(x) == True), 'Offense']= x
        dataset.loc[dataset.Offense == x, 'Forcible']= 'Y'

    return dataset

In [2]:
crime = pd.read_csv('Data/ch_crime.csv')
crime.head()

,X,Y,Incident_ID,Agency,Offense,Street,City,State,Zipcode,Date_of_Report,...,Reported_As,Premise_Description,Forcible,Weapon_Description,Victim_Age,Victim_Race,Victim_Gender,Latitude,Longitude,ObjectId
0,-79.032123,35.932317,74949,CHPD,B&E RESIDENCE-FORCE,1521 E FRANKLIN ST,CHAPEL HILL,NC,27514.0,2010/02/16 21:54:00+00,...,NaN,HOME OF VICTIM - OTHER DWELLING,Y,NaN,29.0,W,F,35.932317,-79.032123,1
1,-79.007788,35.944235,74950,CHPD,SUSPICIOUS PERSON,OLD DURHAM RD/POPE RD,CHAPEL HILL,NC,27514.0,2010/02/16 23:27:00+00,...,NaN,SERVICE/GAS STATION,NaN,NaN,NaN,NaN,NaN,35.944235,-79.007788,2
2,-79.054006,35.925060,74956,CHPD,DRIVING WHILE IMPAIRED,798 MARTIN LUTHER KING JR BLVD/UMSTEAD DR,CHAPEL HILL,NC,27514.0,2010/02/17 02:08:00+00,...,NaN,HIGHWAY/ROAD/STREET ETC.,N,NaN,NaN,NaN,NaN,35.925060,-79.054006,3
3,-79.052878,35.914568,74952,CHPD,LARCENY- ALL OTHER,201 EAST FRANKLIN ST,CHAPEL HILL,NC,27514.0,2010/02/17 02:23:00+00,...,NaN,OTHER RETAIL ESTABLISHMENT,N,NaN,22.0,W,F,35.914568,-79.052878,4
4,-79.061153,35.910799,74951,CHPD,SUSPICIOUS PERSON,409 WEST FRANKLIN ST,CHAPEL HILL,NC,27514.0,2010/02/17 02:41:00+00,...,NaN,RESTAURANT,N,NaN,NaN,NaN,NaN,35.910799,-79.061153,5


In [3]:
#Take out unnecessary variables
cleanSet = crime[['X','Y', 'Offense', 'Date_of_Occurrence', 'Forcible', 'Weapon_Description', 'Victim_Gender', 'Reported_As']]

#Rename X and Y
cleanSet = cleanSet.rename(columns = {'X':'Longitude','Y':'Latitude'})

# split column and add new columns to cleanSet
cleanSet[['Date', 'Time']] = cleanSet['Date_of_Occurrence'].str.split(' ', expand=True)
cleanSet = cleanSet[['Longitude','Latitude', 'Offense', 'Forcible', 'Weapon_Description', 'Date', 'Time', 'Victim_Gender', 'Reported_As']]
cleanSet['Time'] = cleanSet['Time'].str[:-3]

#Removes None Forcible Offenses

In [4]:
crime_labels = ['SEXUAL', 'ASSAULT', 'VANDALISM', 'LARCENY', 'B&E', 'ROBBERY', 'MURDER', 'BURGLARY', 'RAPE']
combine(cleanSet, crime_labels)

cleanSet = cleanSet[cleanSet.Forcible == 'Y']
cleanSet = cleanSet[cleanSet.Victim_Gender.notna()]

len(cleanSet)
#cleanSet.head(5)

17056

In [5]:
#cleanSet.loc[(cleanSet.Offense.str.contains('ROBBERY') == True), 'Offense']= 'ROBBERY'

crime_removal = []

cleanSet.loc[(cleanSet.Offense.str.contains('HOMICIDE') == True), 'Offense']= 'MURDER'

cleanSet = combine(cleanSet, crime_labels)
sum = len(cleanSet[cleanSet.Offense == 'B&E'])
print("B&E      ", sum)
sum = len(cleanSet[cleanSet.Offense == 'ASSAULT'])
print("ASSAULT  ", sum)
sum = len(cleanSet[cleanSet.Offense == 'VANDALISM'])
print("VANDALISM", sum)
sum = len(cleanSet[cleanSet.Offense == 'LARCENY'])
print("LARCENY  ", sum)
sum = len(cleanSet[cleanSet.Offense == 'ROBBERY'])
print("ROBBERY  ", sum)
sum = len(cleanSet[cleanSet.Offense == 'BURGLARY'])
print("BURGLARY ", sum)
sum = len(cleanSet[cleanSet.Offense == 'MURDER'])
print("MURDER   ", sum)
sum = len(cleanSet[cleanSet.Offense == 'SEXUAL'])
print("SEXUAL   ", sum)

B&E       2382
ASSAULT   3117
VANDALISM 1734
LARCENY   7428
ROBBERY   254
BURGLARY  139
MURDER    12
SEXUAL    1016


In [6]:
cleanSet.head()

,Longitude,Latitude,Offense,Forcible,Weapon_Description,Date,Time,Victim_Gender,Reported_As
0,-79.032123,35.932317,B&E,Y,NaN,2010/02/16,16:30:00,F,NaN
3,-79.052878,35.914568,LARCENY,Y,NaN,2010/02/17,02:22:00,F,NaN
7,-79.076591,35.940808,ASSAULT,Y,NaN,2010/02/12,09:00:00,M,NaN
12,-79.020974,35.934196,B&E,Y,NaN,2010/02/17,10:00:00,M,NaN
15,-79.018476,35.952147,VANDALISM,Y,SHOTGUN,2010/02/17,17:39:00,F,NaN


In [7]:
len(cleanSet)

17056

In [8]:
array = cleanSet['Offense'].unique()
array

array(['B&E', 'LARCENY', 'ASSAULT', 'VANDALISM', 'INFORMATION',
       'DAMAGE TO VEHICLE', 'DISTURBING THE PEACE', 'N', 'ROBBERY',
       'AUTOMOBILE THEFT', 'SUSPICIOUS PERSON', 'CALL FOR SERVICE',
       'DISTURBANCE', 'SUSPICIOUS CONDITION', 'SEXUAL', 'BURGLARY',
       'CREDIT CARD/ATM FRAUD', 'STRANGULATION',
       'COMMUNICATING THREATS -INTIMIDATION  NON PHYSICAL THREAT',
       'ASSIST EMS', 'FIGHTING/ AFFRAY', 'UNAUTHORIZED USE OF CONVEYANCE',
       'CALL FOR SERVICE-UNFOUNDED', 'FOUND PROPERTY',
       'DAMAGE TO PROPERTY(NON-CRIMINAL)', 'MISSING PROPERTY',
       'DAMAGE TO PROPERTY',
       'UNDISCIPLINED JUVENILE LESS THAN 16 YEARS/BUT OLDER THAN 6',
       'DOMESTIC DISTURBANCE', 'DAMAGE TO PROPERTY(CRIMINAL)',
       'DOG BITES AND ATTACKS', 'REFUSE TO LEAVE', 'TRESPASSING',
       'HOME INVASION', 'UNDERAGE DRINKING', 'DRIVING WHILE IMPAIRED',
       'DOMESTIC AFFRAY', 'PURSE SNATCHING', 'VEHICLE CRASH',
       'FIGHT/AFFRAY', 'BURNING PERSONNEL PROPERTY',
       'AS

In [9]:
len(array)

161